In [1]:
#Diversos
import pandas as pd
from IPython.display import clear_output
import ToolBox.kit_function as kit

#WebDriver
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

#Ações
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from ast import Break

# Exceções
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException as Timeout
from selenium.common.exceptions import ElementNotInteractableException as NotInteract
from selenium.common.exceptions import ElementClickInterceptedException as NotAvailibleElement
from selenium.common.exceptions import NoSuchWindowException as CrashJanela
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Inicializa a instância do EDGE
service = Service()

options = webdriver.EdgeOptions()
options.add_argument("-inprivate")
driver = webdriver.Edge(service=service, options=options)
driver.maximize_window()

url = 'https://toolsworldb2b.meuspedidos.com.br/'

driver.get(url)

In [3]:
sheetCategorias = 'Links/CategoriasMTX.xlsx'
columns = ['Link']
categoriasLinks = kit.readExcel(sheetCategorias,columns)

In [ ]:
init = 0
for posicao, link in enumerate(categoriasLinks['Link'][init:len(categoriasLinks['Link'])], start=init):
    clear_output(wait=False)
    print(f'Posição: {posicao}, Link: {link}')
    driver.get(link)
    
    #------------------------------Captura o título da categoria------------------------------
    cssSeletorTitle = '#catalogWrapper > div:nth-child(2) > div.Catalog__titleB2bContainer___JtAi4 > div > div.Catalog__titleB2bInner___G8ViQ'

    titleCategoria = kit.safe_find_element(driver,By.CSS_SELECTOR , cssSeletorTitle).text
    print(titleCategoria)
    
    
    #-----------------------Verifica a quantidade de itens na página--------------------------
    cssSeletorMoreItens = "button[class='b2b-button Catalog__links___4450R Catalog__buttonSeeMoreProducts___kZGUm Button__button___Oss6- Button__pointer___mYfJr Button__info___4h9Qg Button__medium___FQv5P Button__buttonInfo___aqiZj custom-brand-full-b2b']"

    #-----------------------Verifica se o item está disponível-----------------------
    kit.VerifyElement(By.CLASS_NAME, 'catalog-item-b2b-title',driver)
    itens = driver.find_elements(By.CLASS_NAME, 'catalog-item-b2b-title')
    print('Iniciando varredura, quantidade inicial:', len(itens))

    #-----------------------Scroll-----------------------
    try:
        while True:
            kit.VerifyElement(By.CSS_SELECTOR, cssSeletorMoreItens,driver)
            verProdutos = driver.find_element(By.CSS_SELECTOR, cssSeletorMoreItens)
            verProdutos.click()
    except:
            print('Verificação Terminada.')
            #Break()
            
    kit.rollpage(driver,By.TAG_NAME, "footer")

    #-----------------------Seta a quantidade de itens na pagina-----------------------
    itens = driver.find_elements(By.CLASS_NAME, 'catalog-item-b2b-title')
    print('Total: ' + str(len(itens)) + ' Itens')
    
    
    #-----------------------Captura dos Dados-----------------------
    #Array de Dados
    nomeList= []
    codList = []
    prcList = []
    impList = []
    prodUrlList = []
    
    #Dados do Produto
    clsNomeProd = 'ProductInfo__productNameWithAddedToCardIcon___CTBpv'
    clsCodProd = 'ProductInfo__productInfo___rlEL5'
    clsPrcProd = 'AddToCartContainer__informationForSale___1DaML'
    cssMaisInfo = "i[class='materialIcons__materialIcons___NHWaT PopoverPriceDetails__icon___r5kg2 PopoverPriceDetails__container___Gki+K']"
    xpathImpostoProd = '/html/body/div[4]/div/ul[4]/li[2]'


    #Variavel contadora de itens pulados
    iJump = 0
    
    #Controle de posição
    prod = 0
    
    #Titulo do produto
    clsProdTitle1 = 'catalog-item-b2b-title'

    #Método para percorrer a página
    while prod < len(itens):
        try:
            #Espera carregar
            kit.VerifyElement(By.CLASS_NAME, clsProdTitle1,driver)
            itens = driver.find_elements(By.CLASS_NAME,clsProdTitle1)
            print('Prox item n°: ' , prod , ' ', itens[prod].text)
            
            #Vai ao item
            driver.execute_script("arguments[0].scrollIntoView(true);", itens[prod])
            #Ajuste fino
            kit.rollpage(driver,By.TAG_NAME, "footer", -350)
            
            itens[prod].click()
            
        except (NotInteract,NotAvailibleElement):
            print('Produto Não Disponivel N°: ',iJump ,'. Incrementando')
            prod += 1
            iJump +=1
            print('Not found:' , itens[prod].text)
            itens[prod].click()
        
        #---------------------------Entrou no Produto----------------------
        
        # Captura o nome 
        nome = kit.safe_find_element(driver, By.CLASS_NAME, clsNomeProd, 'Nome não encontrado')

        # Captura o código
        cod = kit.safe_find_element(driver, By.CLASS_NAME, clsCodProd, 'Código não encontrado')

        # Captura o preço
        prc = kit.safe_find_element(driver, By.CLASS_NAME, clsPrcProd, "erro")

        # Captura o imposto
        try:
            hoverable = driver.find_element(By.CSS_SELECTOR, cssMaisInfo)
            ActionChains(driver).move_to_element(hoverable).perform()
            imp = driver.find_element(By.XPATH, xpathImpostoProd).text
        except:
            imp = "erro"


        if nome == 'Nome não encontrado':
            prod-=1
        else:     
            nomeList.append(nome.text)
            codList.append(cod.text)
            prcList.append(prc.text)
            impList.append(imp)
            prodUrlList.append(driver.current_url)
        
        prod +=1    
        driver.back()

        df = {'Nome':nomeList,
            'codigo': codList,
            'prc': prcList,
            'imp': impList,
            'link':prodUrlList}

        xlsx_dir = 'Dados/'

        mtxTable = pd.DataFrame(df)

        file_name = xlsx_dir + titleCategoria + '.xlsx'

        mtxTable.to_excel(file_name, index=False)
        print('Itens de:' + titleCategoria + ' Salvos')    
        
        
        #print(nomeList)
        #print(codList)
        #print(prcList)
        #print(impList)
        #print(prodUrlList)

Elemento encontrado!!!
Prox item n°:  13   ALICATE CORTE DIAGONAL, REFORCADO, FOSFATIZADO, 6POL.-150MM, 1PC // GROSS
Esperando Elemento...
